In [7]:
# Python의 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# 코드 실행시 경고창 실행 여부 관련
import warnings
warnings.filterwarnings('ignore')


# 사이킷런 관련 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler

### Q. 타이타닉 생존자 예측 데이터 세트 train.csv에 대하여 다음 사항을 수행하세요.
- 일괄 전처리 사용자 함수 transform_features(df) 작성
- dt, lr, rf 모델링 및 평가(정확도)
- dt_clf , folds=5 적용하여 KFold 교차검증 수행
- dt_clf , cv=5 적용, cross_val_score를 이용하여 교차검증 수행
- GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행.
  - parameters = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}
  - dt_clf, scoring='accuracy', cv=5 적용

In [79]:
data_df = pd.read_csv('/content/drive/MyDrive/KITA_OCR과정/m5_머신러닝/datas/train.csv')
data_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [39]:
# [1] 일괄 전처리 사용자 함수 transform_features(df) 작성
from sklearn.preprocessing import LabelEncoder

# 사용자 함수 fillna(df) :::  Null 처리 (대상 항목 : Age, Cabin, Embarked, Fare)
def fillna(df):
  df['Age'].fillna(df['Age'].mean(), inplace=True)      # 'Age'의 Null 값을 평균으로 대체
  df['Cabin'].fillna('N', inplace=True)                 # 'Cabin'의 Null 값을 'N'이라는 문자로 대체
  df['Embarked'].fillna('N', inplace=True)              # 'Embarked'의 Null 값을 'N'이라는 문자로 대체
  df['Fare'].fillna(0, inplace=True)                    # 'Fare'의 Null 값을 '0'이라는 숫자로 대체

  return df


# 사용자 함수 drop_features(df) ::: 머신러닝 알고리즘에 불필요한 속성 제거 (대상 항목 : PassengerId, Name, Ticket(티켓번호))
def drop_features(df):
  df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

  return df


# 사용자 함수 format_features(df) ::: 레이블 인코딩 수행 (대상 항목 : Cabin(선실번호), Sex(성별), Embarked(중간 정착 항구))
def format_features(df):
  df['Cabin'] = df['Cabin'].str[:1]     # 'Cabin'의 데이터 중 첫 문자만 추출

  features = ['Cabin', 'Sex', 'Embarked']

  for feature in features:
    le = LabelEncoder()
    le = le.fit(df[feature])
    df[feature] = le.transform(df[feature])

  return df


# 사용자 함수 transform_features(df) ::: 위 세 개의 함수를 통합적으로 일괄 전처리
def transform_features(df):
  df = fillna(df)
  df = drop_features(df)
  df = format_features(df)

  return df

In [80]:
# [2] dt, lr, rf 모델링 및 평가(정확도)

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 분할 후 학습 및 예측
y_data_df = data_df['Survived']                   # Y 데이터 생성 (원본 데이터에서 y값만 추출)
X_data_df = data_df.drop('Survived', axis=1)      # X 데이터 생성 (y값인 'Survived' 항목 삭제)
X_data_df = transform_features(X_data_df)         # transform_features(df) 함수 적용하여 X 데이터의 전처리 실행

X_train, X_test, y_train, y_test = train_test_split(X_data_df, y_data_df, test_size=0.20, random_state=10)


# dt(Decision Tree, 의사결정 트리) 모델링 및 정확도 확인
dt_clf = DecisionTreeClassifier(criterion='gini', max_depth=4, min_samples_split=10, random_state=12)

dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)                       # 학습이 완료된 객체(dt_clf)를 이용하여 테스트 데이터 세트로 예측 수행.

dt_accuracy = accuracy_score(y_test, dt_pred)
print("[Decision Tree] 데이터 정확도 확인 :", dt_accuracy)


# lr(Logistic Regression, 로지스틱 회귀) 모델링 및 정확도 확인
lr_clf = LogisticRegression(random_state=42)

lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)

lr_accuracy = accuracy_score(y_test, lr_pred)
print("[Logistic Regression] 데이터 정확도 확인 :", lr_accuracy)


# rf(Random Forest, 랜덤 포레스트) 모델링 및 정확도 확인
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

rf_clf.fit(X_train, y_train)

# Predict the response for the test dataset
rf_pred = rf_clf.predict(X_test)

# Calculate and print the accuracy
rf_accuracy = accuracy_score(y_test, rf_pred)
print(f"[Random Forest] 데이터 정확도 확인 : {rf_accuracy:.2f}")

[Decision Tree] 데이터 정확도 확인 : 0.8435754189944135
[Logistic Regression] 데이터 정확도 확인 : 0.8268156424581006
[Random Forest] 데이터 정확도 확인 : 0.83


In [69]:
# 교차검증을 위해 데이터 재준비
features = X_data_df.values         # 전처리된 X 데이터를 array로 변환하여 features 준비 (data_df.to_numpy() 코드를 이용할 수도 있는 듯...)
label = y_data_df.values            # 전처리된 y 데이터를 array로 변환하여 label 준비 (data_df.to_numpy() 코드를 이용할 수도 있는 듯...)

In [68]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

# [3] dt_clf, folds=5 적용하여 KFold 교차검증 수행

# dt_clf는 [2]에서 만든 값 활용  :::  dt_clf = DecisionTreeClassifier(criterion='gini', max_depth=4, min_samples_split=10, random_state=12)

# 5개의 폴드 세트로 분리하는 KFold 객체 & 세트별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits = 5)     # folds=5 적용
cv_accuracy = []

n_iter = 0

# KFold 객체의 split() 호출하면 폴드별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환
for train_index, test_index in kfold.split(features):
  # kfold.split()으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
  X_train, X_test = features[train_index], features[test_index]
  y_train, y_test = label[train_index], label[test_index]

  # 학습 및 예측
  dt_clf.fit(X_train, y_train)
  pred = dt_clf.predict(X_test)
  n_iter += 1

  # 반복시마다 정확도 측정
  accuracy = np.round(accuracy_score(y_test, pred), 4)
  train_size = X_train.shape[0]
  test_size = X_test.shape[0]

  print('\n [# {0}] 교차검증 정확도: {1}  /  학습 데이터 크기: {2}  /  검증 데이터 크기: {3}'.format(n_iter, accuracy, train_size, test_size))
  print(' [# {0}] 검증 세트 인덱스: {1}'.format(n_iter, test_index))
  cv_accuracy.append(accuracy)

# 각각의 iteration별 정확도를 합하여 평균 정확도 계산
print('\n ## 평균 검증 정확도:', np.mean(cv_accuracy))



 [# 1] 교차검증 정확도: 0.8268  /  학습 데이터 크기: 712  /  검증 데이터 크기: 179
 [# 1] 검증 세트 인덱스: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178]

 [# 2] 교차검증 정확도: 0.8034  /  학습 데이터 크기: 713  /  검증 데이터 크기: 178
 [# 2] 검증 세트 인덱스: [179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 2

In [71]:
from sklearn.model_selection import cross_val_score, cross_validate

# [4] dt_clf , cv=5 적용, cross_val_score를 이용하여 교차검증 수행

# 교차검증을 위해 준비된 데이터(features, label 활용)
# dt_clf는 [2]에서 만든 값 활용  :::  dt_clf = DecisionTreeClassifier(criterion='gini', max_depth=4, min_samples_split=10, random_state=12)

# 성능 지표는 정확도(accuracy), 교차 검증 세트는 5개
scores = cross_val_score(dt_clf, features, label, scoring='accuracy', cv=5)

print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.7542 0.8202 0.8034 0.7865 0.8427]
평균 검증 정확도: 0.8014


In [77]:
# [5] GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행.
#     - parameters = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}
#     - dt_clf, scoring='accuracy', cv=5 적용

from sklearn.model_selection import GridSearchCV, train_test_split

# 교차검증을 위해 준비된 데이터(features, label 활용)
# dt_clf는 [2]에서 만든 값 활용  :::  dt_clf = DecisionTreeClassifier(criterion='gini', max_depth=4, min_samples_split=10, random_state=12)
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=121)
dtree = DecisionTreeClassifier(random_state=12)     # random_state = 10 ~ 12 (random_state를 넣지 않아도 변화 없음.)

# parameter들을 dictionary 형태로 설정
parameters = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}

# param_grid의 하이퍼 파라미터들을 5개의 train, test-set fold로 나누어서 테스트 수행 설정 (5개로 나누는 것의 원본 데이터는 교차 검증에 들어오기 전 학습용(train) 데이터를 가지고 나눔.)
grid_search = GridSearchCV(dtree, param_grid=parameters, cv=5, refit=True)     # refit=True가 default (True이면 가장 좋은 파라미터 설정으로 재 학습시킴)

# wine Train 데이터로 param_grid의 하이퍼파라미터들을 순차적으로 학습 및 교차 검증(평가)
grid_search.fit(X_train, y_train)

print("[ 학습 내용 ] \n", grid_search, '\n')

print('GridSearchCV 최적 파라미터:', grid_search.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_search.best_score_))

[ 학습 내용 ] 
 GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 3, 5, 10],
                         'min_samples_leaf': [1, 5, 8],
                         'min_samples_split': [2, 3, 5]}) 

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.8328
